In [1]:
import os
from steam_web_api import Steam

KEY = os.environ.get("STEAM_API_KEY")
steam = Steam(KEY)

terraria_app_id = 105600

# arguments: app_id
user = steam.apps.get_app_details(terraria_app_id)
print(user)

{'105600': {'success': True, 'data': {'type': 'game', 'name': 'Terraria', 'steam_appid': 105600, 'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': [409210, 1323320], 'detailed_description': 'Dig, Fight, Explore, Build:  The very world is at your fingertips as you fight for survival, fortune, and glory.   Will you delve deep into cavernous expanses in search of treasure and raw materials with which to craft ever-evolving gear, machinery, and aesthetics?   Perhaps you will choose instead to seek out ever-greater foes to test your mettle in combat?   Maybe you will decide to construct your own city to house the host of mysterious allies you may encounter along your travels? <br><br>In the World of Terraria, the choice is yours!<br><br>Blending elements of classic action games with the freedom of sandbox-style creativity, Terraria is a unique gaming experience where both the journey and the destination are completely in the player’s control.   The Terraria adventure

In [2]:
import sqlite3
import requests
import tweepy
import os
from dotenv import load_dotenv
import tempfile
from datetime import datetime, timedelta
import pytz

# Charger les variables d'environnement
load_dotenv()

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
PARIS_TZ = pytz.timezone('Europe/Paris')
TIMESTAMP_THRESHOLD = int((datetime.now(PARIS_TZ) - timedelta(days=1)).timestamp())


def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen >= ?
    """, (TIMESTAMP_THRESHOLD,))
    return cursor.fetchall()

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def filter_game(game_data):
    return game_data['type'] == 'game' and not game_data.get('dlc', False)

def get_twitter_api():
    auth = tweepy.OAuthHandler(
        os.getenv('TWITTER_CONSUMER_KEY'),
        os.getenv('TWITTER_CONSUMER_SECRET')
    )
    auth.set_access_token(
        os.getenv('TWITTER_ACCESS_TOKEN'),
        os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
    )
    return tweepy.API(auth)

def get_twitter_client():
    client = tweepy.Client(
        consumer_key=os.getenv('TWITTER_CONSUMER_KEY'),
        consumer_secret=os.getenv('TWITTER_CONSUMER_SECRET'),
        access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
        access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
    )
    return client

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            temp_file.write(response.content)
            return temp_file.name
    return None

def format_tweet_message(game_data):
    try:
        app_id = next(iter(game_data))
        game_info = game_data[app_id]['data']
        
        name = game_info['name']
        genres = ", ".join([genre['description'] for genre in game_info.get('genres', [])])
        developers = ", ".join(game_info.get('developers', []))
        description = game_info.get('short_description', '')
        
        # Construire le lien Steam
        steam_link = f"https://store.steampowered.com/app/{app_id}"
        
        # Construire le tweet avec le lien
        tweet = f"Name: {name}. {genres}. Dev: {developers}. Desc: \"{description}\". Link: {steam_link}"
        
        # Vérifier et ajuster la longueur du tweet si nécessaire
        if len(tweet) > 280:
            # Calculer l'espace disponible pour la description
            available_space = 280 - len(f"Name: {name}. {genres}. Dev: {developers}. Desc: \"...\". Link: {steam_link}")
            truncated_description = description[:available_space] + "..."
            tweet = f"Name: {name}. {genres}. Dev: {developers}. Desc: \"{truncated_description}\". Link: {steam_link}"
        
        return tweet
    except KeyError as e:
        print(f"Erreur lors du formatage du tweet: Clé manquante - {e}")
        return None
    except Exception as e:
        print(f"Erreur inattendue lors du formatage du tweet: {e}")
        return None

def send_tweet(message, image_url=None):
    api = get_twitter_api()
    client = get_twitter_client()
    try:
        media_ids = None
        if image_url:
            image_path = download_image(image_url)
            if image_path:
                uploaded_media = api.media_upload(filename=image_path)
                media_ids = [uploaded_media.media_id]
                os.unlink(image_path)  # Supprimer le fichier temporaire

        # Créer le tweet avec l'image si disponible
        response = client.create_tweet(text=message, media_ids=media_ids)
        return response.data['id']
    except Exception as e:
        print(f"Erreur lors de la création du tweet: {e}")
        return None

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    
    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn)
            
            for steam_game_id, first_seen in new_entries:
                print(f"Nouveau jeu trouvé : Steam ID: {steam_game_id}, First Seen: {first_seen}")
                game_data = get_game_details(steam_game_id)
                if game_data and filter_game(game_data):
                    message = format_tweet_message(game_data)
                    tweet_id = send_tweet(message)
                    if tweet_id:
                        print(f"Tweet publié pour {game_data['name']} (ID: {tweet_id})")
                else:
                    print(f"Le jeu avec Steam ID {steam_game_id} ne répond pas aux critères de tweet ou les détails n'ont pas pu être récupérés.")
            
            conn.close()
        else:
            print("Échec du téléchargement de la base de données")
    
    os.unlink(temp_db.name)  # Supprime le fichier temporaire


#publie un tweet à partir d'un game id
if __name__ == "__main__":
    terraria_app_id = 105600
    game_data = get_game_details(terraria_app_id)
    tweet = format_tweet_message(game_data)
    if tweet:
        print("Tweet formaté :")
        print(tweet)
        
        # Récupérer l'URL de l'image du jeu
        image_url = game_data[str(terraria_app_id)]['data'].get('header_image')
        
        # Envoyer le tweet avec l'image
        tweet_id = send_tweet(tweet, image_url)
        if tweet_id:
            print(f"Tweet publié avec succès. ID: {tweet_id}")
        else:
            print("Échec de la publication du tweet.")
    else:
        print("Échec du formatage du tweet.")

In [ ]:
#next step : réduire le lien steam si possible ?